In [1]:
# pip install --upgrade keras-cv tensorflow

In [2]:
# !pip install tensorflow
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import pandas as pd
import os
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten , Input

2024-12-07 12:21:18.014930: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-07 12:21:18.018158: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-12-07 12:21:18.028037: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733602878.044666  864076 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733602878.049733  864076 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-07 12:21:18.066383: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [3]:
opened = []

csv_path = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/CSV/'

masterlist = '/global/scratch/users/arvalcarcel/CSMUB/RESULTS/ALL_STATIONS_FINAL_REVISED.csv'

stations_df = pd.read_csv(masterlist)
station_num = stations_df['grdc_no']

arrayFile = [os.path.join(csv_path, f"{station_no}.csv") for station_no in station_num]

for file in arrayFile:
  df = pd.read_csv(file, index_col= None, header = 0)
  opened.append(df)

total_df = pd.concat(opened, axis = 0, ignore_index = True)
print(total_df)

       Unnamed: 0     Date       Q       SWE    SWE_scaled
0               0  2018-08  24.526  2.087791  1.809042e+06
1               1  2018-09  31.372  1.835435  1.590379e+06
2               2  2018-10  19.572  1.976332  1.712464e+06
3               3  2018-11   7.349  1.633273  1.415208e+06
4               4  2018-12  13.824  1.782850  1.544815e+06
...           ...      ...     ...       ...           ...
23053          57  2023-05   3.046  3.400061  5.610100e+04
23054          58  2023-06   2.489  3.195970  5.273351e+04
23055          59  2023-07   2.021  3.656259  6.032828e+04
23056          60  2023-08   2.067  3.578097  5.903860e+04
23057          61  2023-09   2.483  3.630205  5.989839e+04

[23058 rows x 5 columns]


In [4]:
q = total_df['Q']
swe = total_df['SWE']
scaled = total_df['SWE_scaled']

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU

# Convert q and swe to NumPy arrays
X = scaled.to_numpy()  # This will be 1D: shape (num_samples,)
y = q.to_numpy()

# Assume X and y are your 1D time series arrays
# Step 1: Replace zeros with a small value
X[X == 0] = 1e-9
y[y == 0] = 1e-9

# Step 2: Reshape data (if needed)
X = X.reshape(-1, 1)
y = y.reshape(-1, 1)

# Step 3: Scale the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y)

In [6]:
# Split into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop, Nadam
from tensorflow.keras.losses import Huber

# Define a Conv1D model
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(128, kernel_size=1, activation='relu', input_shape=(1, 1)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(1)  
])

# Compile the model
model.compile(optimizer=Nadam(learning_rate=1e-3), loss=Huber(delta=0.1), metrics=['mae'])

# Train the model
CNN = model.fit(X_train, y_train, epochs=100, validation_data=(X_val, y_val))

Epoch 1/100


/global/home/users/arvalcarcel/.local/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-12-07 12:21:23.293314: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


577/577 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 307.0329 - mae: 3070.3774 - val_loss: 194.1766 - val_mae: 1941.8170
Epoch 2/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 182.2989 - mae: 1823.0389 - val_loss: 347.6381 - val_mae: 3476.4319
Epoch 3/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 164.5828 - mae: 1645.8776 - val_loss: 104.6897 - val_mae: 1046.9471
Epoch 4/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 122.8997 - mae: 1229.0471 - val_loss: 267.7350 - val_mae: 2677.3997
Epoch 5/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 106.4429 - mae: 1064.4785 - val_loss: 82.5746 - val_mae: 825.7955
Epoch 6/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 104.2354 - mae: 1042.4041 - val_loss: 84.6627 - val_mae: 846.6765
Epoch 7/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 91.2406 - mae: 912.4559 - val_loss: 83.4771 - val_mae: 834.8217
Epoch 8/100
577/577 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 74.4015 - mae: 744.0656 - val_loss: 83.9052 - val_mae: 8

In [8]:
# # Define the model
# CNN = Sequential()
# CNN.add(Input(shape=(1,1)))  # Adjust to match reshaped input
# CNN.add(Conv1D(filters=9332, kernel_size=1, activation='relu'))  # Convolutional layer
# CNN.add(Flatten())  # Flatten for the Dense layers
# CNN.add(Dense(128, activation='relu'))  # Dense layer
# CNN.add(Dense(64, activation='relu'))
# CNN.add(Dense(32, activation='relu'))
# CNN.add(Dense(12, activation='relu')) 
# CNN.add(Dense(1))  # Output layer

# # Compile the model
# CNN.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# # Train the model
# CNNhistory = CNN.fit(X_train, y_train, epochs=200, validation_data=(X_val, y_val))

In [9]:
# # Split into training and validation sets
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# # Define a Conv1D model
# model = tf.keras.Sequential([
#     tf.keras.layers.Conv1D(32, kernel_size=1, activation='relu', input_shape=(1, 1)),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(1)  
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# # Train the model
# CNN = model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))

In [10]:
CNN.summary()

AttributeError: 'History' object has no attribute 'summary'

In [ ]:
# Plot training and validation loss
plt.plot(CNNhistory.history['loss'], label='Training Loss')
plt.plot(CNNhistory.history['val_loss'], label='Validation Loss')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
val_loss, val_accuracy = CNN.evaluate(X_val, y_val)
print(f"Validation Loss: {val_loss}")
print(f"Validation Accuracy: {val_accuracy}")

In [ ]:
plt.plot(CNNhistory.history['accuracy'],'b')
plt.plot(CNNhistory.history['val_accuracy'],'r')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# plt.plot(CNN.history['mae'],'b')
# plt.plot(CNN.history['val_mae'],'r')
# plt.title('Model Mean Absolute Error')
# plt.ylabel('Mean Absolute Error')
# plt.xlabel('Epoch')
# plt.legend(['Train', 'Test'], loc='upper left')
# plt.show()

In [ ]:
# Make predictions on the validation set
y_pred = CNN.predict(X_val)

# Compare predictions with actual values
plt.scatter(y_val, y_pred, alpha=0.5)
plt.title('Predictions vs Actual Values')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

rms = mean_squared_error(y_val, y_pred, squared=False)
print(rms)